In [4]:
from typing import Any, Literal
import random
import math
from pprint import pprint

In [5]:
import numpy as np

class HammingMatrix:
    def __init__(self, m):
        self.m = m
        self.n = 2**m - 1
        self.k = self.n - m
        self._generate_matrices()

    def _generate_matrices(self):
        # Создаем проверочную матрицу H (m x n)
        # Столбцы - двоичные представления чисел от 1 до 2^m - 1
        H = np.array([[(i >> j) & 1 for i in range(1, self.n + 1)] 
                      for j in range(self.m)])
        
        # Для систематического вида переставим столбцы (степени 2 в конец)
        cols_parity = [2**i for i in range(self.m)]
        cols_data = [i for i in range(1, self.n + 1) if i not in cols_parity]
        self.reorder = cols_data + cols_parity
        self.H = H[:, [i-1 for i in self.reorder]]
        
        # Порождающая матрица G = [I_k | P]
        P = self.H[:, :self.k].T
        self.G = np.hstack((np.eye(self.k, dtype=int), P))
        print("self.G")
        print(self.G)
        print("self.H")
        print(self.H)
        print()

    def encode(self, data):
        return np.dot(data, self.G) % 2

    def decode(self, received):
        # Вычисление синдрома S = r * H^T
        syndrome = np.dot(received, self.H.T) % 2
        s_val = int("".join(map(str, syndrome[::-1])), 2)
        
        if s_val == 0:
            return received[:self.k]
        
        # Поиск номера столбца в H, совпадающего с синдромом
        for i in range(self.n):
            if np.array_equal(self.H[:, i], syndrome):
                received[i] ^= 1
                break
        return received[:self.k]


In [6]:
# Пример для m=4 (код 15, 11)
hm = HammingMatrix(m=4)
msg = np.random.randint(0, 2, hm.k)
print(f"{msg} - исходное")
codeword = hm.encode(msg)
print(f"{codeword} - с контрольными битами")
codeword[5] ^= 1 # Ошибка
print(f"{codeword} - ломанная")
decoded_msg = hm.decode(codeword)
print(f"{decoded_msg} - чиним и возвращаем к исходному")


self.G
[[1 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 1 0 1 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 1 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 1 1 1 0]
 [0 0 0 0 1 0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 1 0 0 0 0 1 1 0 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 1 0 0 1 0 1 1]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]]
self.H
[[1 1 0 1 1 0 1 0 1 0 1 1 0 0 0]
 [1 0 1 1 0 1 1 0 0 1 1 0 1 0 0]
 [0 1 1 1 0 0 0 1 1 1 1 0 0 1 0]
 [0 0 0 0 1 1 1 1 1 1 1 0 0 0 1]]

[0 1 1 1 1 1 0 0 1 0 0] - исходное
[0 1 1 1 1 1 0 0 1 0 0 0 1 0 1] - с контрольными битами
[0 1 1 1 1 0 0 0 1 0 0 0 1 0 1] - ломанная
[0 1 1 1 1 1 0 0 1 0 0] - чиним и возвращаем к исходному
